# 23. BeautifulSoup 으로 Webpage 읽어오기

- 23_index.html file 을 webserver 에서 open   

    VS Code --> LiveServer 확장 프로그램 설치 --> Open with Live Server

## Web Scraping

- program 이 browser 인 것 처럼 행동하여 web page 를 access 하고 정보 추출  


    - hard way : regular expression 을 이용  
    - easy way : beautifulsoup 사용
    
- ``pip install beautifulsoup4``

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req
import re

### html file 읽기

- 웹페이지의 주소는 매번 바뀔 수 있으므로 새로운 주소를 아래의 url 에 update 해 줍니다.

In [4]:
# 웹브라우저의 주소창에서 url을 copy 합니다. 
url = "http://localhost:8000/23_index.html"

In [5]:
res = req.urlopen(url)

print(res.read().decode('utf-8'))

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <h1>BeautifulSoup Web Crawling</h1>
    <p>뷰티플숩을 이용한 웹 크롤링</p>
    <div class="firstclass" id="div1">
        <p>첫번째 div 안의 paragraph 입니다.</p>
    </div>
    <div class="secondclass" id="div2">
        <p>두번째 div 안의 paragraph 입니다.</p>
    </div>
</body>
</html>


### BeautifulSoup object 에 html file 저장

In [6]:
res = req.urlopen(url)
soup = BeautifulSoup(res, 'html.parser')

- html 내의 특정 `<div>` tag 검색

In [7]:
soup.find_all('div', id="div1")

[<div class="firstclass" id="div1">
 <p>첫번째 div 안의 paragraph 입니다.</p>
 </div>]

- html 내의 특정 class 검색 

In [8]:
soup.find_all(class_='secondclass')

[<div class="secondclass" id="div2">
 <p>두번째 div 안의 paragraph 입니다.</p>
 </div>]

### soup object 의 내용 읽기

### hard way

- regular expression 이용

#### ``title`` tag 내용 읽기

- 정규식 이용

In [9]:
res = req.urlopen(url)
text = res.read().decode('utf-8')

p1 = re.search('<h1>', text)
p2 = re.search('</h1>', text)

text[p1.span()[1] : p2.span()[0]]

'BeautifulSoup Web Crawling'

- Beautifulsoup 이용

In [10]:
soup.h1.text

'BeautifulSoup Web Crawling'

In [11]:
print(soup.title.get_text())
print(soup.title.text)

Document
Document


#### html 내의 title tag 내용 검색
- 정규식 이용

In [12]:
p1 = re.search('<title>', text)
p2 = re.search('</title>', text)

start = p1.span()[1]
end = p2.span()[0]
text[start: end]

'Document'

- Beautifulsoup 이용

In [13]:
soup.title.get_text()

'Document'

In [14]:
soup.title.text

'Document'

- ``h1`` tag 내용 읽기

In [15]:
print(soup.h1)
print(soup.h1.text)

<h1>BeautifulSoup Web Crawling</h1>
BeautifulSoup Web Crawling


- 같은 tag 가 중복된 경우 첫번째 tag display

In [16]:
soup.div

<div class="firstclass" id="div1">
<p>첫번째 div 안의 paragraph 입니다.</p>
</div>

- 중복된 tag 를 모두 find

In [17]:
soup.find_all('div')

[<div class="firstclass" id="div1">
 <p>첫번째 div 안의 paragraph 입니다.</p>
 </div>,
 <div class="secondclass" id="div2">
 <p>두번째 div 안의 paragraph 입니다.</p>
 </div>]

- 특정 id 를 지정하여 find

In [18]:
print(soup.find_all(id="div1"))
print(soup.find_all(id="div1")[0].text)

[<div class="firstclass" id="div1">
<p>첫번째 div 안의 paragraph 입니다.</p>
</div>]

첫번째 div 안의 paragraph 입니다.



- 특정 class 를 지정하여 find  
- class 는 Python 의 keyword 이므로 class_ 로 표시

In [19]:
soup.find_all(class_="secondclass")

[<div class="secondclass" id="div2">
 <p>두번째 div 안의 paragraph 입니다.</p>
 </div>]

- tag 내의 속성 (attribute) 가져오기

In [20]:
soup.find('div')['class']

['firstclass']

## bitcoin price 읽어 오기

In [22]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

URL = "https://www.coindesk.com/price/bitcoin"
hdr = {'User-Agent': 'Mozilla/5.0'}

- BeautifulSoup object 에 html file 저장

In [23]:
req = Request(URL, headers=hdr)
page = urlopen(req)
html = BeautifulSoup(page, "html.parser")

In [24]:
html

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><script>APP_ENV = "production";</script><script data-cookieconsent="ignore">window.dataLayer = window.dataLayer || [{"app_name":"arc","app_env":"production","app_version":"3.68.4","event":"page_view","content_iso":"BTC","content_language":"en","media_type":"html","page_category":"price_page","page_url":"/price/bitcoin/","page_pathname":"/price/bitcoin/","sponsored":false,"video_provider":"primis"}];function gtag() {window.dataLayer.push(arguments);}gtag("consent", "default", {ad_storage: "granted",analytics_storage: "granted",functionality_storage: "granted",personalization_storage: "granted",security_storage: "granted",wait_for_update: 500});gtag("set", "ads_data_redaction", false);gtag("set", "url_passthrough", true);gtag("consent", "default", {ad_storage: "denied",analytics_storage: "denied",functionality_storage: "granted",personalization_storage: "denie

- page source 보기에서 비트코인 가격 있는 class_ id 찾기

In [26]:
found = html.find(class_='currency-pricestyles__Price-sc-1v249sx-0 jobFak')
found

<span class="currency-pricestyles__Price-sc-1v249sx-0 jobFak">41,010.08</span>

In [27]:
match = re.search('([0-9,.]+)</span>', str(found))
match.group()

'41,010.08</span>'

In [28]:
match.group(1)

'41,010.08'